<a href="https://colab.research.google.com/github/giirrr/first_ryun_project/blob/main/Optuna_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xgboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mlp
import warnings
import sklearn
import optuna
import os
warnings.filterwarnings('ignore')


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error

"""data 불러오기"""
X_train = np.loadtxt('./train_data/X_train.csv', delimiter=',')
X_test = np.loadtxt('./train_data/X_test.csv', delimiter=',')

y_train = np.loadtxt('./train_data/y_train.csv', delimiter=',')
y_val = np.loadtxt('./train_data/y_test.csv', delimiter=',')
"""data 불러오기"""

xgb_model = xgboost.XGBRegressor()
"""최적의 학습모델 찾기"""
num_train = 3
for j in range(num_train): # 학습을 여러번반복해도 동일한 결과가 나왔다.
    xgb_model = xgboost.XGBRegressor( n_estimators=600, learning_rate=0.1, gamma=0, subsample=0.5, colsample_bytree=0.3,
                                     max_depth=5) # 학습 모델 생성
    xgb_model.fit(X_train, y_train)
    xgb_model.save_model("./learn_data/{0}epoch.json".format(j))



from optuna.samplers import TPESampler

sampler = TPESampler(seed=10)


def objective(trial):
    dtrain = xgboost.DMatrix(X_train, label=y_train)
    dtest = xgboost.DMatrix(X_test, label=y_val)

    param = {
        'objective': 'reg:squarederror',
        #'verbose': -1,
        #'metric': 'rmse',

        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),

    }

    model = xgboost.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, eval_set=[(X_test, y_val)], verbose=0, early_stopping_rounds=25)
    rmse = mean_squared_error(y_val, xgb_model.predict(X_test))
    return rmse


study_xgb = optuna.create_study(direction='minimize', sampler=sampler)
study_xgb.optimize(objective, n_trials=100)

trial = study_xgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

#Best Trial: score 6.230401680950585e-06,
#params {'max_depth': 10, 'n_estimators': 1133, 'colsample_bytree': 0.26366023216199885, 'subsample': 0.9404820937429552}
optuna.visualization.plot_optimization_history(study_xgb).show()
optuna.visualization.plot_parallel_coordinate(study_xgb).show()
optuna.visualization.plot_contour(study_xgb).show()